In [2]:
from hello.hello import *

In [3]:
alarms = [
    "%s_Low_Low%s",
    "%s_Low%s",
    "%s_High%s",
    "%s_High_High%s",
]


def mk_alrm(grp, unit, templates):
    if unit:
        unit = "_(%s)" % unit
    return [t % (grp, unit) for t in templates]



In [13]:
from time import sleep


def test_alarm(app, alrm, pv, offset):
    app.login()
    app.setconfig("Process_Alarms", alrm, pv + offset)
    
    
def restore_alarm(app, alrm, cfg):
    app.login()
    app.setconfig("Process_Alarms", alrm, cfg[alrm])

def test_alarm_group(app, alrm_grp, unit, mv_func, cfg):
    alrms = mk_alrm(alrm_grp, unit, alarms)
    offsets = (20, 10, -10, -20)
    exp_code = 90, 40, 50, 100
    tests = zip(alrms, offsets, exp_code)
    
    for a, o, c in tests:
        while mv_func()['error'] != 0: 
            sleep(0.1)
        
        num_alarms = app.getUnAckCount()
        test_alarm(app, a, mv_func()['pv'], o)
        
        while mv_func()['error'] == 0 or app.getUnAckCount() <= num_alarms: 
            sleep(0.1)
        
        e = mv_func()['error']
        res = 'pass' if c == e else 'fail'
        print(a, 'error:', e, "...", res)
        restore_alarm(app, a, cfg)
        
    print("Done")

def announce_set_config(app, alrm, cfg):
    app.login()
    print("\rSetting config:", a, cfg[a], "                ",end="")
    app.setconfig('Process_Alarms', a, cfg[a])
    
def test_grp(app, grp, unit, mv_grp, cfg):
    for a in mk_alrm(grp, unit, alarms):
        announce_set_config(app, a, cfg)
    print("\r                                  \r", end="")
    f = lambda: app.gpmv()[mv_grp]
    test_alarm_group(app, grp, unit, f, cfg)

In [11]:
def run_test(app, groups, cfg):
    for g, u, mvg in groups:
        print("Testing", g)
        test_grp(app, g, u, mvg, cfg)

In [9]:
groups = (
     ('Agitation', 'RPM', 'agitation'),
     ('DO', '%', 'do'),
     ('Filter_Oven', 'C', 'condenser'),
     ('Pressure', 'psi', 'pressure'),
     ('Temp', 'C', 'temperature'),
     ('Total_Gas', 'LPM', 'maingas'),
     ('pH', '', 'ph'),
     ('Level', 'L', 'level')
)
stuff = ("Agitation", "RPM"), ("DO", "%"), ("Filter_Oven", "C"),\
        ("Pressure", "psi"), ("Temp", "C"), ("Total_Gas", "LPM"),\
        ("pH", "")
alrms = [mk_alrm(g, u, alarms) for g, u in stuff]
app = open_hello('71.189.82.196:6')
cfg = app.getConfig()['Process_Alarms']
assert all(k in cfg for g in alrms for k in g)

In [15]:
run_test(app, groups, cfg)

Testing Level
Level_Low_Low_(L) error: 90 ... pass
Level_Low_(L) error: 40 ... pass
Level_High_(L) error: 50 ... pass
Level_High_High_(L) error: 100 ... pass
Done
Testing Total_Gas
Total_Gas_Low_Low_(LPM) error: 90 ... pass
Total_Gas_Low_(LPM) error: 40 ... pass
Total_Gas_High_(LPM) error: 50 ... pass
Total_Gas_High_High_(LPM) error: 100 ... pass
Done
